In [ ]:
import os
import sys
import glob
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import RandomState, SeedSequence, MT19937
if '../DL' not in sys.path:
    sys.path.append('../DL')
from utils import *

### Functions that define the animal position and a place cell's firing rate

In [ ]:
def pos(t, speed, length):
    return (speed * t) % length

def time_constant(x, middle_PF, sigma_PF):
    return np.exp(-(x - middle_PF)**2 / (2 * (sigma_PF/(2*np.pi)*track_length)**2))

def firing_rate(t, max_rate, length_PF, start_PF, middle_PF, sigma_PF, animal_speed, track_length, ftheta):
    x = pos(t, animal_speed, track_length)
    rate = max_rate * time_constant(x, middle_PF, sigma_PF) * \
                np.cos(2 * np.pi * ftheta * t + \
                       np.pi / length_PF * (x - start_PF))
    if np.isscalar(t):
        return rate if rate > 0 else 0
    rate[rate < 0] = 0
    return rate

#### Simulation parameters taken from the authors' code, not necessarily from the paper

In [ ]:
track_length = 300 # (cm)
animal_speed = 32.43567842 # (cm/s)
lap_time = track_length / animal_speed # (s)
max_rate = 20 # (Hz)
length_PF = 30 # (cm)
r = track_length / (2 * np.pi)
phi_PF_rad = length_PF / (2 * np.pi)
middle_PF = track_length * np.random.uniform() # (cm)
start_PF = (middle_PF - length_PF / 2) % track_length
# sigma_PF = 0.146
sigma_PF = length_PF/2 / track_length * 2 * np.pi / 3
ftheta = 7 # (Hz)
total_time = 3600 # (s)

print(f'Beginning of the place field: {start_PF:.3f} cm')
print(f'Middle of the place field: {middle_PF:.3f} cm')
print(f'Stddev of the place field: {sigma_PF:.3f} rad')
print(f'Time to run a single lap: {lap_time:.2f} s')

#### Plot animal position and place cell's firing rate as a function of time

In [ ]:
n_laps = 1
t = np.linspace(0, n_laps * lap_time, n_laps * 1000, endpoint=False)
x = pos(t, animal_speed, track_length)
tau = time_constant(x, middle_PF, sigma_PF)
rate = firing_rate(t, max_rate,
                   length_PF, start_PF, middle_PF, sigma_PF,
                   animal_speed, track_length, ftheta)

fig,ax = plt.subplots(3, 1, figsize=(6, 6), sharex=True)
ax[0].plot(t, x, 'k', lw=1)
ax[1].plot(t, tau, 'k', lw=1)
ax[2].plot(t, rate, 'k', lw=1)
ax[0].set_xlim([0, lap_time])
ax[0].set_ylabel('Position (cm)')
ax[1].set_ylabel(r'$\tau$ (s)')
ax[2].set_ylabel('Firing rate (spikes/s)')
ax[2].set_xlabel('Lap time (s)')
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

## Generation of inhomogeneous Poisson spike times

We start by generating `n_laps` indipendent trials, each with a duration equal to a single lap time.

In [ ]:
rate_fun = lambda t: firing_rate(t, max_rate,
                                 length_PF, start_PF, middle_PF, sigma_PF,
                                 animal_speed, track_length, ftheta)
n_laps = int(total_time / lap_time)
seed = int(time.time())
rs = RandomState(MT19937(SeedSequence(seed)))
seeds = np.unique(rs.randint(0, 100000, size=2 * n_laps))[:n_laps]
states = [RandomState(MT19937(SeedSequence(seed))) for seed in seeds]
spikes, all_spikes = [], []
for state in states:
    a,b = make_inhomogeneous_poisson_spike_train(rate_fun, max_rate,
                                                 tend=lap_time,
                                                 random_state=state,
                                                 full_output=True)
    spikes.append(a)
    all_spikes.append(b)

When we plot these, we see that the firing rate of the inhomogeneous Poisson process perfectly matches the theoretical firing rate of the place cell: this happens because each trial is aligned to time 0.

In [ ]:
pos_fun = lambda times: [pos(t, animal_speed, track_length) for t in times]
fig,ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
rasterplot(pos_fun(all_spikes), 50, ax=ax[0])
rasterplot(pos_fun(spikes), 50, ax=ax[0], marker='x', color='r', markersize=3)
bw = 0.1 / ftheta
nu, edges, count = psth(all_spikes, binwidth=bw, interval=[0, lap_time])
ax[1].plot(pos_fun(edges), nu, 'k', lw=1, label='Hom Poisson rate')
nu, edges, count = psth(spikes, binwidth=bw, interval=[0, lap_time])
ax[1].plot(pos_fun(edges), rate_fun(edges), 'g', lw=1, label='Theor rate')
ax[1].plot(pos_fun(edges), nu, 'm', lw=1, label='Inhom Poisson rate')
ylim = ax[1].get_ylim()
ax[1].plot(start_PF + np.zeros(2), ylim, 'c--', lw=1)
ax[1].plot(middle_PF + np.zeros(2), ylim, 'c--', lw=1)
ax[1].plot(middle_PF + length_PF / 2 + np.zeros(2), ylim, 'c--', lw=1)
ax[0].set_xlim([0, track_length])
ax[0].set_ylabel('Lap #')
ax[1].set_ylabel('Firing rate (spikes/s)')
ax[1].set_xlabel('Position (cm)')
ax[1].legend(loc='best')
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

We now generate a single realization, lasting `total_time`, and then reshape it based on the duration of each lap.

In [ ]:
seed = int(time.time())
rs = RandomState(MT19937(SeedSequence(seed)))
spikes, all_spikes = make_inhomogeneous_poisson_spike_train(rate_fun, max_rate,
                                                            tend=total_time,
                                                            random_state=rs,
                                                            full_output=True)
edges = np.r_[0 : total_time : lap_time]
jdx = np.digitize(all_spikes, edges)
all_spikes_reshaped = [all_spikes[jdx == j] - (j-1) * lap_time for j in np.unique(jdx)]
jdx = np.digitize(spikes, edges)
spikes_reshaped = [spikes[jdx == j] - (j-1) * lap_time for j in np.unique(jdx)]

Now we see that the place cell's firing rate does not match perfectly the theoretical one: this has to do with the fact that laps are not perfectly aligned as before (set `animal_speed = 30` above to obtain a firing rate that looks exactly like the one in the previous plot). This also matches what we see applying the same procedure to the spike times generated by the paper authors' code.

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
max_laps = 50
rasterplot(pos_fun(all_spikes_reshaped), max_laps, ax=ax[0])
rasterplot(pos_fun(spikes_reshaped), max_laps, ax=ax[0], marker='x', color='r', markersize=3)
nu, edges, count = psth(all_spikes_reshaped, binwidth=0.1 / ftheta, interval=[0, lap_time])
ax[1].plot(pos_fun(edges), nu, 'k', lw=1, label='Hom Poisson rate')
nu, edges, count = psth(spikes_reshaped, binwidth=0.1 / ftheta, interval=[0, lap_time])
ax[1].plot(pos_fun(edges), rate_fun(edges), 'g', lw=1, label='Theor rate')
ax[1].plot(pos_fun(edges), nu, 'm', lw=1, label='Inhom Poisson rate')
ylim = ax[1].get_ylim()
ax[1].plot(start_PF + np.zeros(2), ylim, 'c--', lw=1)
ax[1].plot(middle_PF + np.zeros(2), ylim, 'c--', lw=1)
ax[1].plot(middle_PF + length_PF / 2 + np.zeros(2), ylim, 'c--', lw=1)
ax[0].set_xlim([0, track_length])
ax[0].set_ylabel('Lap #')
ax[1].set_ylabel('Firing rate (spikes/s)')
ax[1].set_xlabel('Position (cm)')
ax[1].legend(loc='best')
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

#### Data generated with the paper's code

In [ ]:
data_folder = os.path.join('..', 'files')
place_cell_ratio = 0.5
n_neurons = 8000
linear = True
f_in = make_filename('spike_trains', n_neurons, place_cell_ratio, total_time, linear, '.npz')
spikes_file = os.path.join(data_folder, f_in)
f_in = make_filename('PFstarts', n_neurons, place_cell_ratio, total_time, linear, '.pkl')
PF_pos_file = os.path.join(data_folder, f_in)
spike_trains = np.load(spikes_file, allow_pickle=True)['spike_trains']
PF_positions = pickle.load(open(PF_pos_file, 'rb'))

# this is the value in the code that generates these spike times
animal_speed = 32.43567842 # (cm/s)
lap_time = track_length / animal_speed # (s)

In [ ]:
idx = 1002
spks = np.array(spike_trains[idx])
edges = np.r_[0 : total_time : lap_time]
jdx = np.digitize(spks, edges)
tmp = [spks[jdx == j] - (j-1) * lap_time for j in np.unique(jdx)]
nu, edges, count = psth(tmp, binwidth=0.1 / ftheta, interval=[0, lap_time])
fig,ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
max_laps = 50
rasterplot(pos_fun(tmp), max_laps, ax=ax[0], color='r')
ax[1].plot(pos_fun(edges), nu, 'm', lw=1, label='Inhom Poisson rate')
if idx in PF_positions:
    x = PF_positions[idx] / (2 * np.pi) * track_length
    ax[0].plot(x + np.zeros(2), [0, max_laps], 'g--')
    ax[1].plot(x + np.zeros(2), [0, nu.max()], 'g--', label='PF start')
ax[0].set_xlim([0, track_length])
ax[1].set_xlabel('Position (cm)')
ax[0].set_ylabel('Lap #')
ax[1].set_ylabel('Firing rate (spikes/s)')
ax[1].legend(loc='best')
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()